In [ ]:
%load_ext autoreload
%autoreload 2
import math
from pathlib import Path

import h5py
import numpy as np
from scipy import constants, signal, stats, optimize
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cycler import cycler

import analysis
from basic.paths import (
    RESULTS_FOLDER,
    PARTICLE_VARIATION_FOLDER,
    DENSITY_VARIATION_FOLDER,
    V_FLOW_VARIATION_FOLDER,
    FOLDER_1D,
    MPLSTYLE_FILE
)
from basic import (
    physics,
    Species,
    SpeciesInfo,
    RunInfo
)

from plots import (
    settings,
    plots_1D,
    plots_2D,
)

info = RunInfo(
    electron=SpeciesInfo(
        number_density=12.0e6,
        temperature=100.0,
        charge=-1,
        mass=1.0,
        bulk_velocity=0.0
    ),
    proton=SpeciesInfo(
        number_density=10.0e6,
        temperature=3.0,
        charge=+1,
        mass=1836.152674,
        bulk_velocity=0.0
    ),
    alpha=SpeciesInfo(
        number_density=1.0e6,
        temperature=12.0,
        charge=+2,
        mass=7294.29953,
        bulk_velocity=1.0e5
    )
)

save=False
plt.style.use(MPLSTYLE_FILE)
matplotlib.rcParams['figure.dpi'] = settings.FIGURE_DPI

# Simulation 2D

## Alpha flow-speed variation (B=0)

In [ ]:
plots_2D.maxEnergyVsAlphaFlowSpeed(info)
plt.show()
plots_2D.maxEnergyNormalizedVsAlphaFlowSpeed(info)
plt.show()

In [ ]:
plots_2D.waveAngleVsAlphaFlowSpeed(info, "x")
plots_2D.simulatedAlphaFlowSpeed(info, "x")
plt.show()

TODO: |k| vs u-alpha

In [ ]:
plots_2D.omegaVsAlphaFlowSpeed(info, "x")
plots_2D.omegaVsAlphaFlowSpeed(info, "y")
plt.show()

In [ ]:
plots_2D.psdOmegaForAlphaFlowSpeed(info, "x")
plots_2D.psdOmegaForAlphaFlowSpeed(info, "y")
plt.show()

In [ ]:
for species in Species:
    plots_2D.temperature3DOverTimeForAlphaFlowSpeed(
        info, species
    )

In [ ]:
for species in Species:
    plots_2D.temperatureDifferences3DVsAlphaFlowSpeed(info, species)
plt.show()

In [ ]:
for data_file in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.electricField2DSnapshot(data_file, info, time=50.0)
    plt.show()

In [ ]:
for data_file in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.energyEFieldOverTime(data_file, info)
    plt.show()

In [ ]:
for data_file in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.strengthBFieldOverTime(data_file, info)
    plots_2D.psdBField(data_file, info)
    plots_2D.energyBField(data_file, info)
    plt.show()

In [ ]:
for species in Species:
    plots_2D.psdFlowVelocity(info, species, "x", "x")
    plt.show()

In [ ]:
for species in Species:
    plots_2D.flowVelocityVsTime(info, species, "x", "x")
    plt.show()

In [ ]:

for data_file in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plt.figure()
    for species in Species:
        u_alpha = int(data_file.stem[-3:])
        with h5py.File(data_file) as f:
            time = f["Header/time"][:] * info.omega_pp
            x_grid = f[f"Grid/x_px/{species.value}/X"][:]
            px_grid = f[f"Grid/x_px/{species.value}/Px"][:]
            py_grid = f[f"Grid/x_py/{species.value}/Py"][:]
            dist_x_px = f[f'dist_fn/x_px/{species.value}'][:]
            dist_x_py = f[f"dist_fn/x_py/{species.value}"][:]
            T = f[f"Derived/Temperature/{species.value}"][:]
        u_x = analysis.flowVelocity1D(x_grid, px_grid, dist_x_px, info[species])
        u_y = analysis.flowVelocity1D(x_grid, py_grid, dist_x_py, info[species])
        K = 0.5 * info[species].si_mass * np.mean(u_x ** 2 + u_y ** 2, axis=1) / constants.electron_volt
        U = 3 / 2 * physics.kelvinToElectronVolt(np.mean(T, axis=(1,2)))
        W_species = info[species].number_density * (K + U)
        plt.plot(time, (W_species - W_species[0]) * 1e-6, label=f"$W_{{{species.symbol()}}}$")
    with h5py.File(data_file) as f:
        time = f["Header/time"][:] * info.omega_pp
        E_x = f['Electric Field/ex'][:]
        E_y = f['Electric Field/ey'][:]
    W_E = np.mean(E_x ** 2 + E_y ** 2, axis=(1,2)) * (constants.epsilon_0 / 2) / constants.electron_volt
    plt.plot(time[1:], (W_E[1:] - W_E[1]) * 1e-6, label="$W_E$")
    # plt.yscale("log")
    plt.xlabel("Time $t\\,\\omega_{pp}$ (1)")
    plt.ylabel("Energy density W (MeV$\\,/\\,$m$^3$)")
    plt.legend()
    plt.show()

# Videos of distribution

In [ ]:
# from scipy import ndimage
data_file = RESULTS_FOLDER / "epoch_2D/v_alpha_bulk_variation/v_alpha_bulk_106.h5"
with h5py.File(data_file) as f:
    time = f["Header/time"][:] * info.omega_pp
    x_grid = f["Grid/grid/X"][:]
    y_grid = f["Grid/grid/Y"][:]
    px_grid = f["Grid/px_py/Protons/Px"][:]
    py_grid = f["Grid/px_py/Protons/Py"][:]
    dist_p_grid = f['dist_fn/px_py/Protons'][:]
    print(f['dist_fn/px_py/Protons'].attrs["units"])
# plt.imshow(full_dist.T, norm="log")
# plt.colorbar()
# plt.contour(ndimage.gaussian_filter(full_dist.T, sigma=4.0, order=0), levels=10 ** np.linspace(5.5, 7.8, 4), colors="black", linewidths=2, linestyles=":")
# plt.show()
v_x = px_grid / (info.proton.mass * constants.electron_mass)
v_y = py_grid / (info.proton.mass * constants.electron_mass)
dv_x = v_x[1] - v_x[0]
dv_y = v_y[1] - v_y[0]
length_x = abs(x_grid[-1] - x_grid[0])
length_y = abs(y_grid[-1] - y_grid[0])
f = dist_p_grid / (dv_x * dv_y * length_x * length_y) # s^2/m^4

In [ ]:
dist = np.trapezoid(y=f, x=v_y, axis=2)
fig = plt.figure()
line = plt.plot(v_x * 1e-3, dist[-1], color="black", lw=3)[0]
text = plt.text(0.72, 0.9, s=f"t$\\,=\\,${time[0]:>5.1f}", transform=plt.gca().transAxes)
plt.yscale("log")
plt.xlabel("Velocity $v_{\\alpha,x}$ (km/s)")
plt.ylabel("$\\langle f_\\alpha\\rangle_\\mathbf{x}$ (s/m$^3$)")
# plt.xlim(0, 400)
# plt.xlim(-100, 100)
# plt.xlim(0, 300)
# plt.ylim(1e-3, 4e1)

def update(frame_idx):
    line.set_ydata(dist[2 * frame_idx])
    text.set_text(f"t$\\,\\omega_{{pp}}\\,=\\,${time[2 * frame_idx]:>5.1f}")
    return (line, text,)

# ani = animation.FuncAnimation(fig=fig, func=update, frames=range(150, 500))
# ani.save(filename="figures/distribution_vx_alpha=180.mp4", writer="ffmpeg", fps=60)
plt.show()

In [ ]:
dist = dist_p_grid / (dx * dy)
dist[dist <= 0] = np.min(dist[dist>0])
vx_bounds = np.min(v_x) + np.arange(v_x.size + 1) * dv_x - dv_x / 2
vy_bounds = np.min(v_y) + np.arange(v_y.size + 1) * dv_y - dv_y / 2
fig = plt.figure()
# plt.imshow(dist[0].T)
quad = plt.pcolormesh(vx_bounds * 1e-3, vy_bounds * 1e-3, dist[0].T, norm="log")
plt.colorbar(label="# particles / meter (1/m$^2$)")
text = plt.text(130, 80, s=f"t$\\,\\omega_{{pp}}\\,=\\,${time[0]:>5.1f}")
plt.xlabel("Velocity $v_{\\alpha,x}$ (km/s)")
plt.ylabel("Velocity $v_{\\alpha,y}$ (km/s)")
plt.xlim(0, 200)
plt.ylim(-100, 100)
plt.yticks(np.linspace(-100, 100, 5))
# plt.ylim(1e6, 5e9)

def update(frame_idx):
    quad.set_array(dist[2 * frame_idx].T)
    text.set_text(f"t$\\,\\omega_{{pp}}\\,=\\,${time[2 * frame_idx]:>5.1f}")
    return (quad, text,)

ani = animation.FuncAnimation(fig=fig, func=update, frames=range(150, 500))
ani.save(filename="figures/distribution_vx_vy_alpha.mp4", writer="ffmpeg", fps=60)
plt.show()

# Simulation 1D

## Density variation

In [ ]:
for filename in sorted(DENSITY_VARIATION_FOLDER.glob("density_*.h5")):
    print(filename)
    ratio = 10 ** float(filename.stem.split("_")[-1])
    n_electron = 12e6
    n_proton = n_electron / (1 + 2 * ratio)
    n_alpha = ratio * n_proton
    info = RunInfo(
        electron=SpeciesInfo(
            number_density=12.0e6,
            temperature=100.0,
            charge=-1,
            mass=1.0,
            bulk_velocity=0.0
        ),
        proton=SpeciesInfo(
            number_density=n_proton,
            temperature=3.0,
            charge=+1,
            mass=1836.152674,
            bulk_velocity=0.0
        ),
        alpha=SpeciesInfo(
            number_density=n_alpha,
            temperature=12.0,
            charge=+2,
            mass=7294.29953,
            bulk_velocity=1.0e5
        )
    )
    plots_1D.energyEFieldOverTime(
        data_file=filename, info=info
    )
    plt.show()

## Run n=8192

In [ ]:
plots_1D.electricFieldOverSpaceAndTime(
    data_file=FOLDER_1D / "proton-alpha-instability-1D.h5",
    info=info, save=False
)
plt.show()

In [ ]:
plots_1D.averageTemperature3DOverTime(
    data_file=FOLDER_1D / "proton-alpha-instability-1D.h5",
    info=info, save=save
)
plt.show()

In [ ]:
for s in Species:
    plots_1D.velocityDistributionOverTime(
        data_file=FOLDER_1D / "proton-alpha-instability-1D.h5",
        info=info, species=s, save=save
    )
plt.show()

In [ ]:
plots_1D.energyEFieldOverTime(
    data_file=FOLDER_1D / "proton-alpha-instability-1D.h5",
    info=info, save=save
)
plt.show()

## Particle variation

In [ ]:
variation_folder = PARTICLE_VARIATION_FOLDER
for particle_folder in sorted(variation_folder.glob("particles_*")):
    plots_1D.multiElectricFieldEnergyOverTime(
        particle_folder, info, save=save,
        identifer=f"particles_per_cell_{int(particle_folder.stem[-4:])}"
    )

In [ ]:
plots_1D.particleVariationEnergyVsTime(info=info, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationGrowthRate(info, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperature3D(species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXDiff(info, species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXVsTime(info, species, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationWavenumber(info=info, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationFrequency(info=info, save=False)
plt.show()